# Heatmap plot of values from microtiter plates

This example shows how to create a heatmap plot of values from microtiter plates using the `heatmap` function of `matplotlib`/'seaborn'. 
The data is taken from a microtiter plate with 96 wells, and the values are represented in a 2D array format. The heatmap will display the values in a color-coded format, where higher values are represented by darker colors. 

The data is read from an external source with the labDataReader Library, and the heatmap is generated using the `seaborn` library. The x-axis represents the columns of the microtiter plate, while the y-axis represents the rows. The color bar on the right indicates the scale of values.


Dataframe created by labDataReader is named 'abs_df' with columns: 'well', 'time', 'absorbance'

Example data structure:
```
 abs_df = pd.DataFrame({
     'well': ['A1', 'A2', ..., 'H12'],
     'datetime': [t1, t2, ..., tn],
     'value': [val1, val2, ..., valn]
 })```


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from labdatareader.data_reader import DataReader, read_data

In [ ]:
# Create a custom colormap from green to red
colors = ["green", "yellow", "red"]
cmap = LinearSegmentedColormap.from_list("value", colors)

In [ ]:
abs_data_dir = "./data"
filename_pattern = "bcSP0484_20231010_173832_HFA_20231010_Varioskan_SPabs_600.xml"

In [ ]:
filename_pattern = "bcSP0484_20231010_173832_HFA_20231010_Varioskan_SPabs_600.xml"
skanit7_dr = read_data(
        method="absorbance",
        data_format="thermo_skanit7.uv_vis",
        data_path=abs_data_dir,
        filename_pattern=filename_pattern,
        #metadata_default_core=core_metadata_default,
    )

In [ ]:
abs_df = skanit7_dr.dataframe
abs_df

In [ ]:
# Get unique row and column values directly from dataframe
# Sorted to ensure proper ordering (A-H, 1-12)
rows = sorted(abs_df['row'].unique())
cols = sorted(abs_df['column'].unique())

In [ ]:
# Pivot the data to create a matrix for the heatmap

# Using the last measurement time (or max/mean as needed)
# Using last measurement (can change to max/mean as needed)
#  Reindex to ensure all wells are represented (fills missing with NaN)

heatmap_data = (abs_df.groupby(['row', 'col'])['value']
                .last()  # or .max() or .mean()
                .unstack(level=1)
                .reindex(index=rows, columns=cols, fill_value=np.nan))

heatmap_data


In [ ]:
# Create the heatmap
plt.figure(figsize=(12, 8))
ax = sns.heatmap(heatmap_data, 
                annot=True, 
                fmt=".2f", 
                cmap=cmap,
                cbar_kws={'label': 'Absorbance'},
                linewidths=0.5,
                linecolor='gray',
                square=True)

# Customize the plot
plt.title('96-Well Plate Absorbance Measurements', fontsize=16)
plt.xlabel('Column', fontsize=12)
plt.ylabel('Row', fontsize=12)

# Improve tick labels
# ax.set_xticks([x + 0.5 for x in range(len(cols))])
# ax.set_xticklabels(cols)
# ax.set_yticks([y + 0.5 for y in range(len(rows))])
# ax.set_yticklabels(rows)

plt.tight_layout()  # Prevent label cutoff
plt.show()

## Code for auto colour scaling

Auto-Scaling Options Explained:

* Min-Max Scaling (Method 1)

    - Simplest approach

    - Uses exact data range

    - Sensitive to outliers

* Percentile Scaling (Method 2) - Recommended

    - Uses 5th-95th percentiles

    - More robust to outliers

    - Shows 90% of your data range

* Standard Deviation Scaling (Method 3)

    - Good for normally distributed data

    - Shows mean ± 2 standard deviations

    - Automatically excludes extreme values



In [ ]:
# Get data values for scaling
values = abs_df['value'].values
clean_values = values[~np.isnan(values)]  # Remove NaN values for scaling

# Automatic scaling parameters
if len(clean_values) > 0:
    # Method 1: Simple min-max scaling
    vmin = np.min(clean_values)
    vmax = np.max(clean_values)
    
    # Method 2: Percentile-based scaling (robust to outliers)
    vmin = np.percentile(clean_values, 5)  # 5th percentile
    vmax = np.percentile(clean_values, 95)  # 95th percentile
    
    # Method 3: Mean ± 2SD (for normally distributed data)
    # mean = np.mean(clean_values)
    # std = np.std(clean_values)
    # vmin = max(0, mean - 2*std)  # Don't go below 0 for absorbance
    # vmax = mean + 2*std
else:
    vmin, vmax = 0, 1  # Default range if no valid data

# Create heatmap with auto-scaling
plt.figure(figsize=(12, 8))
heatmap_data = (abs_df.groupby(['row', 'col'])['value']
               .last()
               .unstack(level=1))

ax = sns.heatmap(heatmap_data,
                annot=True,
                fmt=".2f",
                cmap=cmap,
                vmin=vmin,
                vmax=vmax,
                cbar_kws={'label': 'Absorbance'},
                linewidths=0.5,
                linecolor='gray',
                square=True)

# Add color range information to title
plt.title(f'96-Well Plate Absorbance (Range: {vmin:.2f}-{vmax:.2f})', fontsize=16)
plt.xlabel('Column', fontsize=12)
plt.ylabel('Row', fontsize=12)

# Add center value marker to colorbar
cbar = ax.collections[0].colorbar
midpoint = (vmin + vmax) / 2
cbar.ax.plot([0, 1], [midpoint]*2, 'w', linewidth=2)

plt.tight_layout()
plt.show()
